In [1]:
import sys
sys.path.append("../")
import numpy as np
import matplotlib.pyplot as plt
import scripts.prediction_utils as predict_u
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_plot_function as theodo_p
import scripts.groundtruth_utils as theodo_g
import scripts.theodolite_function as theodo_f
import time
import torch
from numpy import linalg
import importlib
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
theodo_f = importlib.reload(theodo_f)
predict_u = importlib.reload(predict_u)

In [2]:
## Path of main data files
type_file_input = "CSV"    ## CSV or ROSBAG

path = [
    "../data/20220224/",
]

## Parameters about noise
num_samples = 1000
## Range noise
random_noise_range = [0, 0.004/2, 2]   ## Mean, sigma, ppm,  4mm + 2ppm (2 sigma)  ISO17123-3
## Angles noise
random_noise_angle = [0, 0.000024241/5*4/2]    # Mean, sigma, 5"=0.000024241 precision datasheet  (2 sigma)  ISO17123-3
## Tilt compensator noise
random_noise_tilt = [0, 0.000002424/2]    # Mean, sigma, 0.5"=0.000002424 precision datasheet  (2 sigma)  ISO17123-3
## Weather conditions
weather_data_path = "../data/weather_2022/Quebec/"
data_weather = np.array(theodo_u.read_weather_data(weather_data_path+"data_sorted_2022.txt"))
## Choice of noise model
model_chosen = [1, 1, 0, 0, 0]  # Tilt noise, Atmospheric correction, extrinsic calibration

limit_time = 3  # Max time between 2 measurements to compute the velocity

save_raw_data = True   # If saving the data

In [3]:
%matplotlib notebook
theodo_u = importlib.reload(theodo_u)
theodo_g = importlib.reload(theodo_g)

for file_i in path:
    print("File processed: ", file_i)

    sub_path = "uncertainty/raw_data/"
    P1 = theodo_u.read_raw_data_uncertainty(file_i+sub_path+"prism1.csv")
    P2 = theodo_u.read_raw_data_uncertainty(file_i+sub_path+"prism2.csv")
    P3 = theodo_u.read_raw_data_uncertainty(file_i+sub_path+"prism3.csv")
    t1,t2,t3,d1,d2,d3,a1,a2,a3,e1,e2,e3 = ([] for i in range(12))
    for p1_i, p2_i, p3_i in zip(P1, P2, P3):
        t1.append(p1_i[0])
        t2.append(p2_i[0])
        t3.append(p3_i[0])
        d1.append(p1_i[1])
        d2.append(p2_i[1])
        d3.append(p3_i[1])
        a1.append(p1_i[2])
        a2.append(p2_i[2])
        a3.append(p3_i[2])
        e1.append(p1_i[3])
        e2.append(p2_i[3])
        e3.append(p3_i[3])
    t1 = np.array(t1)
    t2 = np.array(t2)
    t3 = np.array(t3)
    d1 = np.array(d1)
    d2 = np.array(d2)
    d3 = np.array(d3)
    a1 = np.array(a1)
    a2 = np.array(a2)
    a3 = np.array(a3)
    e1 = np.array(e1)
    e2 = np.array(e2)
    e3 = np.array(e3)

    Index1, Speed1 = theodo_g.compute_speed(t1, d1, a1, e1, num_samples, random_noise_range, random_noise_angle, random_noise_tilt, data_weather, model_chosen)
    Index2, Speed2 = theodo_g.compute_speed(t2, d2, a2, e2, num_samples, random_noise_range, random_noise_angle, random_noise_tilt, data_weather, model_chosen)
    Index3, Speed3 = theodo_g.compute_speed(t3, d3, a3, e3, num_samples, random_noise_range, random_noise_angle, random_noise_tilt, data_weather, model_chosen)

    if save_raw_data:
        theodo_u.save_raw_data_uncertainty_speed(t1[Index1], d1[Index1], e1[Index1], a1[Index1], Speed1, file_i+sub_path+"speed_prism1.csv")
        theodo_u.save_raw_data_uncertainty_speed(t2[Index2], d2[Index2], e2[Index2], a2[Index2], Speed2, file_i+sub_path+"speed_prism2.csv")
        theodo_u.save_raw_data_uncertainty_speed(t3[Index3], d3[Index3], e3[Index3], a3[Index3], Speed3, file_i+sub_path+"speed_prism3.csv")

File processed:  ../data/20220224/
Conversion done !
Conversion done !
Conversion done !
